In [2]:
import os

from IDRnD.utils import Train, seed_everything
from IDRnD.augmentations import ToMellSpec, PadOrClip, ToTensor, Normalize_predef
from IDRnD.dataset import Test_Dataset, get_train_data
from IDRnD.resnet import resnet18

import numpy as np
import pandas as pd
import librosa
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import transforms

seed_everything(0)

In [2]:
dataset_dir = "."

eval_protocol_path = "protocol_test.txt"
eval_protocol = pd.read_csv(eval_protocol_path, sep=" ", header=None)
eval_protocol.columns = ['path', 'key']
eval_protocol['score'] = 0.0
eval_protocol['path'] = eval_protocol['path'].apply(lambda x: os.path.join(dataset_dir, x))

FileNotFoundError: [Errno 2] File b'protocol_test.txt' does not exist: b'protocol_test.txt'

In [3]:
post_transform = transforms.Compose([
    ToMellSpec(n_mels=128),
    Normalize_predef(-29.6179, 16.6342),
    librosa.power_to_db,
    PadOrClip(150),
    ToTensor(),
])

### predict

In [4]:
X, y = get_train_data()

In [ ]:
test_dataset = Test_Dataset(X)

In [ ]:
lens = [len(test_dataset[i][0]) for i in range(len(test_dataset))]

In [ ]:
mda = pd.DataFrame()
mda["path"] = X
mda["lens"] = lens

In [ ]:
pd.set_option('max_colwidth', 80)

In [ ]:
%matplotlib inline
mda["lens"].hist(bins = 500)

In [ ]:
wavs = [test_dataset[i][0] for i in range(len(test_dataset))]

In [ ]:
from hashlib import sha1

hashes = []
for wav in wavs:
    hashes.append(sha1(wav).hexdigest())

In [1]:
mda["hashes"]=hashes

NameError: name 'hashes' is not defined

In [ ]:
mda.groupby("hashes")

In [ ]:
len(set(hashes))

In [ ]:
from collections import Counter
common_hashes = [i[0] for i in Counter(hashes).most_common(592)]

In [ ]:
common_hashes

In [ ]:
set(mda["lens"])

In [7]:
hm = Train()

#test_dataset = Test_Dataset(eval_protocol["path"], post_transform)
test_dataset = Test_Dataset(X[:520], post_transform)

test_loader = DataLoader(test_dataset, batch_size=250, shuffle=False)

model = resnet18(num_classes=1).cuda()

#model.load_state_dict(torch.load('models/simple_old_conv.pt'))
#model_dst = torch.nn.DataParallel(model, device_ids=[0, 1]).cuda()
#torch.save(model_dst.module.state_dict(),  'models/kaggle2_nonparallel.pt') 
model.eval()
model.load_state_dict(torch.load('models/resnet_18_last_epoch.pt'))
pred = hm.predict_on_test(test_loader, model)

In [8]:
model(test_dataset[0][0].reshape(1, 1, 128, 150).cuda())

tensor([[1.9516e-06]], device='cuda:0', grad_fn=<SigmoidBackward>)

In [9]:
mda = pd.DataFrame()
mda["path"] = X[:520]

In [10]:
mda["score"] = pred.values

In [6]:
eval_protocol["score"] = pred.values
eval_protocol[['path', 'score']].to_csv('answers.csv', index=None)

NameError: name 'eval_protocol' is not defined

In [12]:
!jupyter nbconvert --to script predict.ipynb

[NbConvertApp] Converting notebook predict.ipynb to script
[NbConvertApp] Writing 1657 bytes to predict.py


In [ ]:
mda